In [1]:
from nceiDatabaseConnector.nceiDatabasePackage.nceiDataManager import NCEIDataManager
from nceiDatabaseConnector.nceiDatabasePackage.nceiDatabaseManager import NCEIDatabaseManager

In [2]:
years = [1949, 1950]
stationsFilePath = "./data/stations/"
modifiedStationsFilePath = "./data/stations/modifiedStations/"
downloadCSVFilePath = "./data/NCEI/ghcn/daily/"
modifiedCSVFilePath = "./data/NCEI/modified/daily/"
dbname = "NCEIDatabase"
dbuser = "ESDP"
dbpassword = "esdp1"


In [3]:
#This command writes the dbname, dbuser and dbpassword to an env file which is used by the docker-container during postgresql setup
envVariables = {
    "DATABASE_NAME": dbname,
    "DATABASE_USER": dbuser,
    "DATABASE_PASSWORD": dbpassword,
}

with open(".env", "w") as env_file:
    for key, value in envVariables.items():
        env_file.write(f"{key}={value}\n")

print(".env file created successfully.")


.env file created successfully.


In [6]:
#!docker-compose up -d --build
!docker-compose up -d

time="2024-07-15T00:38:27+02:00" level=warning msg="C:\\Users\\maoka\\git\\ESDP1\\ESDP1-Database-Project\\docker-compose.yml: `version` is obsolete"
 Container esdp1-database-project-db-1  Created
 Container esdp1-database-project-db-1  Starting
 Container esdp1-database-project-db-1  Started


In [9]:
# run this to stop the docker container (but keep the data in the database for later)
!docker-compose stop
# run this to stop and remove the docker container (you will loose all the data within the database and will have to set it up again)
#!docker-compose down

time="2024-07-15T00:53:15+02:00" level=warning msg="C:\\Users\\maoka\\git\\ESDP1\\ESDP1-Database-Project\\docker-compose.yml: `version` is obsolete"
 Container esdp1-database-project-db-1  Stopping
 Container esdp1-database-project-db-1  Stopped


In [5]:
host = "localhost"
port = "5432"

#this defines the names of the columns in the postgresql database
stationCols = ["id", "latitude", "longitude", "elevation", "state", "name", "gsn_flag", "hcn_crn_flag", "wmo_id"]
weatherCols = ["id", "stationcode", "datelabel", "param", "value", "mflag", "qflag", "sflag", "time"]


In [6]:
dataManager = NCEIDataManager()
nceiDatabaseManager = NCEIDatabaseManager(db_name=dbname, db_user=dbuser, db_password=dbpassword, db_host=host,
                                          db_port=port, weather_cols=weatherCols, station_cols=stationCols)

In [7]:
dataManager.download_stations(stationsFilePath)

Data downloaded. Will be saved as ghcnd-stations.txt in ./data/stations/
Data downloaded. Will be saved as readme.txt in ./data/stations/
Data downloaded and saved in ./data/stations/


In [10]:
dataManager.convert_stations(stationsFilePath, modifiedStationsFilePath)

Saved the modified stations file to ./data/stations/modifiedStations//modified_stations.csv


In [11]:
dataManager.download_years(years, downloadCSVFilePath)

...Downloading data from year 1949....
Data downloaded. Will be saved as 1949.csv.gz
Data from year 1949 downloaded and saved.
...Downloading data from year 1950....
Data downloaded. Will be saved as 1950.csv.gz
Data from year 1950 downloaded and saved.


In [12]:
dataManager.export_downloaded_years(years, downloadCSVFilePath, modifiedCSVFilePath)

...Year 1949 processing...
Loading data from year 1949
Data from year 1949 loaded.
Export of year 1949 finished.
...Year 1950 processing...
Loading data from year 1950
Data from year 1950 loaded.
Export of year 1950 finished.


In [14]:
file_path = f"{modifiedStationsFilePath}modified_stations.csv"
nceiDatabaseManager.insert_copy(file_path, "Station", stationCols)

Copying file ./data/stations/modifiedStations/modified_stations.csv to database Station
Insert with copy of file ./data/stations/modifiedStations/modified_stations.csv to table: Station done.
Row count after insertion in table Station: 127994


In [7]:
nceiDatabaseManager.drop_table("Climate1949")
nceiDatabaseManager.drop_table("Climate1950")

Connected to database NCEIDatabase with user ESDP
Table Climate1949 dropped successfully.
Disconnected
Connected to database NCEIDatabase with user ESDP
Table Climate1950 dropped successfully.
Disconnected


In [8]:
for year in years:
    filePath = f"{modifiedCSVFilePath}modified_{year}.csv"
    tableName = nceiDatabaseManager.create_climate_table(year=year)
    nceiDatabaseManager.multi_threaded_insert(file_path=filePath, table_name=tableName, columns=weatherCols,
                                              num_threads=4)


Connected to database NCEIDatabase with user ESDP
Table Climate1949 created successfully.
Disconnected
Total Number of lines: 18089253
Chunk size is 4522314
Splitting ./data/NCEI/modified/daily/modified_1949.csv into 4 chunks of size 4522314.
Writing chunk 0 file to ./data/NCEI/modified/daily/modified_1949_chunk0.csv with size 4522314
Writing chunk 1 file to ./data/NCEI/modified/daily/modified_1949_chunk1.csv with size 4522314
Writing chunk 2 file to ./data/NCEI/modified/daily/modified_1949_chunk2.csv with size 4522314
Writing chunk 3 file to ./data/NCEI/modified/daily/modified_1949_chunk3.csv with size 4522311
Thread started for chunk: ./data/NCEI/modified/daily/modified_1949_chunk0.csv
Thread started for chunk: ./data/NCEI/modified/daily/modified_1949_chunk1.csv
Thread started for chunk: ./data/NCEI/modified/daily/modified_1949_chunk2.csv
Thread started for chunk: ./data/NCEI/modified/daily/modified_1949_chunk3.csv
Connected to database NCEIDatabase with user ESDP
Connected to databa